# Lesson 2: Neural Net Computations

In the previous assignment we used Keras to train a neural network. In this assignment you will build your own minimal neural net library. The basic structure is given to you; you will need to fill in details such as weight updating for backpropogation. Then you will test the network on learning the XOR function.

Read through the class definitions below first to understand the basic architecture.

Then you should add code as necessary where marked "TODO" in the code below and remove the NotImplementedError exceptions.

In [1]:
import numpy as np

## Define a Neural Network Class

In [2]:
class NNet():
    """Implements a basic feedforward neural network."""
    
    def __init__(self):
        self._layers = []  # An ordered list of layers. The first layer is the input; the final is the output.
    
    def _add_layer(self, layer):
        if self._layers:
            # Update pointers. We keep a doubly-linked-list of layers for convenience.
            prev_layer = self._layers[-1]
            prev_layer.set_next_layer(layer)
            layer.set_prev_layer(prev_layer)
            
        self._layers.append(layer)
    
    def add_input_layer(self, size, **kwargs):
        assert type(size).__name__ == 'int', ('Input layer requires integer size. Type was %s instead.' 
                                              % type(size).__name__)
        layer = InputLayer(size=size, **kwargs)
        self._add_layer(layer)

    def add_dense_layer(self, size, **kwargs):
        assert type(size).__name__ == 'int', ('Dense layer requires integer size. Type was %s instead.' 
                                              % type(size).__name__)
        # Find the previous layer's size.
        prev_size = self._layers[-1].size()
        layer = DenseLayer(shape=(prev_size, size), **kwargs)
        self._add_layer(layer)

    def summary(self, verbose=False):
        """Prints a description of the model."""
        for i, layer in enumerate(self._layers):
            print('%d: %s' % (i, str(layer)))
            if verbose:
                print('weights:', layer.get_weights())
                if layer._use_bias:
                    print('bias:', layer._bias)
                print()

        
    def predict(self, x):
        """Given an input vector x, run it through the neural network and return the output vector."""
        assert isinstance(x, np.ndarray)
        
        # Here, x is a vector corresponding to the ith row of the X_data matrix
        # To run x through the network, iterate over the layers, passing the output of the current layer
        # as input to the next layer
        inputs = x
        for i,layer in enumerate(self._layers):
            # Pass inputs through current layer
            outputs = layer.feed_forward(inputs)          
            # If more layers to go, set outputs to be inputs for next layer
            if layer != self._layers[-1]:
                inputs = outputs          
        
        return outputs
        
        
    def train_single_example(self, X_data, y_data, learning_rate=0.01):
        """Train on a single example. X_data and y_data must be numpy arrays."""
        
        assert isinstance(X_data, np.ndarray)
        assert isinstance(y_data, np.ndarray)

        # Forward propagation.
        # Here, we need to get the current output using the predict() method
        output = self.predict(X_data)
            
        # Compare output to y_data
        error = output - y_data
        
        # Backpropagation.
        # Here, we need to walk back through each layer, computing and returning the deltas,
        # which get passed to the next layer upstream
        for layer in list(reversed(self._layers)):
            # Update weights for current layer and return deltas
            deltas = layer.backpropagate(error, learning_rate)
            # Deltas become error for next layer
            error = deltas


    def train(self, X_data, y_data, learning_rate, num_epochs, randomize=True, verbose=True, print_every_n=100):
        """Both X_data and y_data should be ndarrays. One example per row.
        
        This function takes the data and learning rate, and trains the network for num_epochs passes over the 
        complete data set. 
        
        If randomize==True, the X_data and y_data should be randomized at the start of each epoch. Of course,
        matching X,y pairs should have matching indices after randomization, to avoid scrambling the dataset.
        (E.g., a set of indices should be randomized once and then applied to both X and y data.)
        
        If verbose==True, will print a status report every print_every_n epochs with these
        results:
        
        * Results of running "predict" on each example in the training set
        * MSE (mean squared error) on the dataset
        * Accuracy on the dataset
        """
        assert isinstance(X_data, np.ndarray)
        assert isinstance(y_data, np.ndarray)
        assert X_data.shape[0] == y_data.shape[0]

        for e in range(1,num_epochs+1):
            # If verbose is true, print results if epoch mod print_every_n is zero
            if verbose and e % print_every_n == 0:
                print('At epoch %i, will print results' % e)
                print_results = True
            else:
                print_results = False
            # If randomization is requested, shuffle indices
            # Otherwise, just use X and y as is
            if randomize:
                ind = list(range(X_data.shape[0]))
                # Numpy random shuffle method works in place
                np.random.shuffle(ind)
                X_data_e = X_data[ind]
                y_data_e = y_data[ind]
            else:
                X_data_e = X_data
                y_data_e = y_data

            # Iterate over each row in X_data
            for i in range(X_data_e.shape[0]):
                # Make sure and use 'epoch' version, which may or may not be scrambled
                x = X_data_e[i,:]
                y = y_data_e[i]

                # Print prediction results if applicable
                if print_results:
                    print('  Result of calling predict with x = %s -> %f' % (x, self.predict(x)))

                # Train network using current row and target
                self.train_single_example(x,y,learning_rate)
            
            # After training on entire data set, print MSE and accuracy
            if print_results:
                print('  MSE = %f' % self.compute_mean_squared_error(X_data, y_data))
                print('  Accuracy = %f' % self.compute_accuracy(X_data, y_data))
        
    
    def compute_mean_squared_error(self, X_data, y_data):
        """Given input X_data and target y_data, compute and return the mean squared error."""
        assert isinstance(X_data, np.ndarray)
        assert isinstance(y_data, np.ndarray)
        assert X_data.shape[0] == y_data.shape[0]
        
        mse = 0
        # MSE is sum of squared deltas between true and predicted values, divided by the number of samples
        for i in range(X_data.shape[0]):
            x = X_data[i,:]
            output = self.predict(x)
            mse += (y_data[i] - output)**2

        # Divide by number of samples
        mse /= X_data.shape[0]
        
        return mse
    
    def compute_accuracy(self, X_data, y_data):
        """Given input X_data and target y_data, convert outputs to binary using a threshold of 0.5
        and return the accuracy: # examples correct / total # examples."""
        assert isinstance(X_data, np.ndarray)
        assert isinstance(y_data, np.ndarray)
        assert X_data.shape[0] == y_data.shape[0]
        
        correct = 0
        for i in range(len(X_data)):
            outputs = self.predict(X_data[i])
            outputs = outputs > 0.5
            if outputs == y_data[i]:
                correct += 1
        acc = float(correct) / len(X_data)
        return acc

## Define activation functions

In [3]:
class Activation():  # Do not edit; update derived classes.
    """Base class that represents an activation function and knows how to take its own derivative."""
    def __init__(self, name):
        self.name = name
    
    def activate(x):
        """x is a scalar or a numpy array. Returns the output y, the result of applying the function to input x."""
        raise NotImplementedError()
    
    def derivative_given_y(self, y):
        """y is a scalar or a numpy array. 
        
        Returns the derivative d(f)/dx given the *activation* value y."""
        raise NotImplementedError()

In [4]:
class IdentityActivation(Activation):
    """Activation function that passes input through unchanged."""
    
    def __init__(self):
        super().__init__(name='Identity')
    
    def activate(self, x):
        """x is a scalar or a numpy array. Returns the output y, the result of applying the function to input x."""
        return x
    
    def derivative_given_y(self, y):
        """y is a scalar or a numpy array. 
        
        Returns the derivative d(f)/dx given the *activation* value y."""
        return 1
    
    
class SigmoidActivation(Activation):
    """Sigmoid activation function."""

    def __init__(self):
        super().__init__(name='Sigmoid')
    
    def activate(self, x):
        """x is a scalar or a numpy array. Returns the output y, the result of applying the function to input x."""

        # Return input passed through sigmoid activation (y = 1/(1+e^(-x)))
        return 1/(1+np.exp(-x))
    
    def derivative_given_y(self, y):
        """y is a scalar or a numpy array. 
        
        Returns the derivative d(f)/dx given the *activation* value y."""
        # Return derivative of sigmoid function, if y = simga(z), y' = y(1-y)
        return y*(1-y)


## Define a method to initialize neural net weights

In [5]:
def WeightInitializer():
    """Function to return a random weight. for example, return a random float from -1 to 1."""
    # Return a random sample from the uniform distribution [0,1)
    weight = np.random.random()
    # Create another random variable to assign sign, expands random interval to (-1,1)
    sign = np.random.random()
    if sign < 0.5:
        weight = -weight
        
    return weight

## Define a neural net Layer base class

In [6]:
class Layer():
    """Base class for NNet layers. DO NOT MODIFY THIS CLASS. Update derived classes instead.
    
    Conceptually, in this library a Layer consists at a high level of:
      * a collection of weights (a 2D numpy array)
      * the output nodes that come after the weights above
      * the activation function that is applied to the summed signals in these output nodes
      
    So a Layer isn't just nodes -- it's weights as well as nodes.
      
    Specifically, to send signal forward through a 3-layer network, we start with an Input Layer that does
    very little.  The outputs from the Input layer are simply the fed-in input data.  
    
    Then, the next layer will be a Dense layer that holds the weights from the Input layer to the first hidden
    layer and stores the activation function to be used after doing a product of weights and Input-Layer
    outputs.
    
    Finally, another Dense layer will hold the weights from the hidden to the output layer nodes, and stores
    the activation function to be applied to the final output nodes.
    
    For a typical 1-hidden layer network, then, we would have 1 Input layer and 2 Dense layers.
    
    Each Layer also has funcitons to perform the forward-pass and backpropagation steps for the weights/nodes
    associated with the layer.
    
    Finally, each Layer stores pointers to the pervious and next layers, for convenience when implementing
    backprop.
    """
   
    def __init__(self, shape, use_bias, activation_function=IdentityActivation, weight_initializer=None, name=''):
        # These are the weights from the *previous* layer to the current layer.
        self._weights = None
        
        # Tuple of (# inputs, # outputs) for Dense layers or just a scalar for an input layer.
        assert type(shape).__name__ == 'int' or type(shape).__name__ == 'tuple', (
            'shape must be scalar or a 2-element tuple')
        if type(shape).__name__ == 'tuple':
            assert len(shape)==2, 'shape must be 2-dimensional. Was %d instead' % len(shape)
        self._shape = shape 
    
        # True to use a bias node that inputs to each node in this layer; False otherwise.
        self._use_bias = use_bias
        
        if use_bias:
            bias_size = shape[-1] if len(shape) > 1 else shape
            self._bias = np.zeros(bias_size)
            if weight_initializer:
                for i in range(bias_size):
                    self._bias[i] = weight_initializer()
        
        # Activation function to be applied to each dot product of weights with inputs.
        # Instantiate an object of this class.
        self._activation_function = activation_function() if activation_function else None
        
        # Method used to initialize the weights in this Layer at creation time.
        self._weight_initializer = weight_initializer
        
        # Layer name (optional)
        self._name = name
        
        # Calculated output vector from the most recent feed_forward(inputs) call.
        self._outputs = None
        
        # Doubly linked list pointers to neighbor layers.
        self._prev_layer = None  # Previous layer is closer to (or is) the input layer.
        self._next_layer = None  # Next layer is closer to (or is) the output layer.
    
    def set_prev_layer(self, layer):
        """Set pointer to the previous layer."""
        self._prev_layer = layer
    
    def set_next_layer(self, layer):
        """Set pointer to the next layer."""
        self._next_layer = layer
    
    def size(self):
        """Number of nodes in this layer."""
        if type(self._shape).__name__ == 'tuple':
            return self._shape[-1]
        else:
            return self._shape
        
    def get_weights(self):
        """Return a numpy array of the weights for inputs to this layer."""
        return self._weights
    
    def get_bias(self):
        """Return a numpy array of the bias for nodes in this layer."""
        return self._bias
    
    def feed_forward(self, inputs):
        """Feed the given inputs through the input weights and activation function, and set the outputs vector.
        
        Also returns the outputs vector for convenience."""
        raise NotImplementedError()
        
    def backpropagate(self, error, learning_rate):
        """Adjusts the weights coming into this layer based on the given output error vector.
        
        For the output layer, the "error" vector should be a list of output errors, y_k - t_k.
        For a hidden layer, the "error" vector should be a list of the delta values from the following layer, such as delta_z_k
        
        Returns a list of the delta values for each node in this layer. These deltas can be used as the error
        values when calling backpropagate on the previous layer."""
        raise NotimplementedError()
        
    def __str__(self):
        activation_fxn_name = self._activation_function.name if self._activation_function else None
        return '[%s] shape %s, use_bias=%s, activation=%s' % (self._name, self._shape, self._use_bias,
                                                              activation_fxn_name)

### Define InputLayer and DenseLayer base classes

The DenseLayer class is where most of the computation happens

In [7]:
class InputLayer(Layer):
    """A neural network 1-dimensional input layer."""
    
    def __init__(self, size, name='Input'):
        assert type(size).__name__ == 'int', 'Input size must be integer. Was %s instead' % type(size).__name__
        super().__init__(shape=size, use_bias=False, name=name, activation_function=None)
    
    def feed_forward(self, inputs):
        assert len(inputs)==self._shape, 'Inputs must be of size %d; was %d instead' % (self._shape, len(inputs))
        self._outputs = inputs
        return self._outputs

    def backpropagate(self, error, learning_rate):
        return None  # Nothing to do.

In [8]:
class DenseLayer(Layer):
    """A neural network layer that is fully connected to the previous layer."""
    
    def __init__(self, shape, use_bias=True, name='Dense', **kwargs):
        super().__init__(shape=shape, use_bias=use_bias, name=name, **kwargs)
        
        self._weights = np.zeros(shape)
        if self._weight_initializer:
            for i in range(shape[0]):
                for j in range(shape[1]):
                    self._weights[i,j] = self._weight_initializer()
    
    def feed_forward(self, inputs):
        """Feed the given inputs through the input weights and activation function, and set the outputs vector.
        
        Also returns the outputs vector for convenience."""
        assert len(inputs)==self._shape[0], 'Inputs must be of size %s; was %s instead' % (self._shape, len(inputs))
        
        num_nodes = self.size()
        
        output = np.zeros(num_nodes)
        # For each node, compute dot product of input values and corresponding column of weight tensor
        # The weight tensor is of size (input nodes, layer nodes)
        for i in range(num_nodes):
            x = np.dot(inputs,self._weights[:,i])
            # Add bias term if applicable
            if self._use_bias:
                x += self._bias[i]
            # Pass through activation function
            output[i] = self._activation_function.activate(x)
        
        # Update output vector for later use, and return it.
        self._outputs = output 
        return self._outputs
        
    def backpropagate(self, error, learning_rate):
        """Adjusts the weights coming into this layer based on the given output error vector.
        
        For the output layer, the "error" vector should be a list of output errors, y_k - t_k.
        For a hidden layer, the "error" vector should be a list of the delta values from the following layer, 
        such as delta_z_k
        
        Returns a list of the delta values for each node in this layer. These deltas can be used as the error
        values when calling backpropagate on the previous layer."""
        assert isinstance(error, np.ndarray)
        assert isinstance(self._prev_layer._outputs, np.ndarray)
        assert isinstance(self._outputs, np.ndarray)  
        
        # Compute deltas. 
        deltas = None
        # One delta term for each node
        num_nodes = self.size()
        deltas = np.zeros(num_nodes)
        for i in range(num_nodes):
            # Differentiate between output layer and general layer
            if not self._next_layer:
                # For output layer, delta is entry from error vector multiplied by 
                # the derivative of the activation function given the output of the layer
                # For a sigmoid activation function, this is equal to (y_k - t_k)*y_k*(1-y_k)
                deltas[i] = error[i] * self._activation_function.derivative_given_y(self._outputs[i])
            else:
                # For a hidden layer, delta is summation of delta values from following layer multiplied
                # by the weights from the current node to each node of that layer, multiplied by
                # the derivative of the activation function given the output of the layer
                deltas[i] = np.dot(error,self._next_layer._weights[i,:]) * self._activation_function.derivative_given_y(self._outputs[i])
                
        # Compute gradient.
        # The number of terms in the gradient is determined by the number of weights
        # Need one gradient per weight term
        gradient = np.zeros(self._shape)
        for i in range(gradient.shape[0]):
            for j in range(gradient.shape[1]):
                gradient[i,j] = deltas[j]*self._prev_layer._outputs[i]
        
        # Adjust weights.
        # Subtract gradient multiplied by the learning rate
        self._weights -= learning_rate * gradient
        
        # Adjust bias weights.
        if self._use_bias:
            # For deltas, inputs are always equal to one so the gradient is just the deltas
            # Subtract gradient multiplied by the learning rate
            self._bias -= learning_rate * deltas
            
        return deltas

# Train a neural net

## Create a dataset for the XOR problem

In [9]:
X_data = np.array([[0,0],[1,0],[0,1],[1,1]])
y_data = np.array([[0,1,1,0]]).T
print(X_data)
print(y_data)

[[0 0]
 [1 0]
 [0 1]
 [1 1]]
[[0]
 [1]
 [1]
 [0]]


## Create a neural network using the library.

In [10]:
nnet = NNet()
nnet.add_input_layer(2)
nnet.add_dense_layer(2, weight_initializer=WeightInitializer, activation_function=SigmoidActivation)
nnet.add_dense_layer(1, weight_initializer=WeightInitializer, activation_function=SigmoidActivation, name='Output')
nnet.summary()

0: [Input] shape 2, use_bias=False, activation=None
1: [Dense] shape (2, 2), use_bias=True, activation=Sigmoid
2: [Output] shape (2, 1), use_bias=True, activation=Sigmoid


In [11]:
nnet.summary(verbose=True)

0: [Input] shape 2, use_bias=False, activation=None
weights: None

1: [Dense] shape (2, 2), use_bias=True, activation=Sigmoid
weights: [[ 0.02423262 -0.54017482]
 [-0.33705096 -0.29935796]]
bias: [-0.17972313  0.19412596]

2: [Output] shape (2, 1), use_bias=True, activation=Sigmoid
weights: [[ 0.03471281]
 [-0.87833632]]
bias: [-0.77801281]



# Train the network

In [12]:
learning_rate = 0.1
num_epochs = 10000
nnet.train(X_data, y_data, learning_rate, num_epochs, randomize=True, verbose=True, print_every_n=2000)

At epoch 2000, will print results
  Result of calling predict with x = [0 1] -> 0.531558
  Result of calling predict with x = [0 0] -> 0.376932
  Result of calling predict with x = [1 1] -> 0.573031
  Result of calling predict with x = [1 0] -> 0.541426
  MSE = 0.223262
  Accuracy = 0.750000
At epoch 4000, will print results
  Result of calling predict with x = [0 1] -> 0.719427
  Result of calling predict with x = [1 0] -> 0.728869
  Result of calling predict with x = [1 1] -> 0.433323
  Result of calling predict with x = [0 0] -> 0.166791
  MSE = 0.090874
  Accuracy = 1.000000
At epoch 6000, will print results
  Result of calling predict with x = [1 1] -> 0.135471
  Result of calling predict with x = [0 1] -> 0.898208
  Result of calling predict with x = [0 0] -> 0.083708
  Result of calling predict with x = [1 0] -> 0.898092
  MSE = 0.011487
  Accuracy = 1.000000
At epoch 8000, will print results
  Result of calling predict with x = [1 0] -> 0.933282
  Result of calling predict with

## Print the resuting neural net weights.

In [13]:
nnet.summary(verbose=True)

0: [Input] shape 2, use_bias=False, activation=None
weights: None

1: [Dense] shape (2, 2), use_bias=True, activation=Sigmoid
weights: [[-3.9338023  -5.69306726]
 [-3.94033196 -5.72939672]]
bias: [5.80457566 2.1527555 ]

2: [Output] shape (2, 1), use_bias=True, activation=Sigmoid
weights: [[ 7.6353909 ]
 [-7.96431195]]
bias: [-3.49408942]



### Trials To Establish 100% Convergence Rate

Note that the network does not always achieve 100% accuracy on the XOR data set (the same was seen with the Keras library from the last assignment). It is worth exploring the rate at which 100% accuracy is achieved. The following block of code runs creates 100 different models using the same architecture as above and stores the accuracy achieved after each training.

In [21]:
def run_trials(num_hidden_nodes,learning_rate,num_epochs,num_trials):
    accuracy = np.zeros(num_trials)
    for i in range(num_trials):
        if (i+1) % 10 == 0:
            print('%3.0f%% of trials complete' % ((i+1)/num_trials * 100))
        nnet_trial = NNet()
        nnet_trial.add_input_layer(2)
        nnet_trial.add_dense_layer(num_hidden_nodes, weight_initializer=WeightInitializer, activation_function=SigmoidActivation)
        nnet_trial.add_dense_layer(1, weight_initializer=WeightInitializer, activation_function=SigmoidActivation, name='Output')

        nnet_trial.train(X_data, y_data, learning_rate, num_epochs, randomize=True, verbose=False)

        accuracy[i] = nnet_trial.compute_accuracy(X_data,y_data)

    return accuracy

In [22]:
# Run trials
num_hidden_nodes = 2
learning_rate = 0.1
num_epochs = 10000
num_trials = 100
accuracy = run_trials(num_hidden_nodes,learning_rate,num_epochs,num_trials)

 10% of trials complete
 20% of trials complete
 30% of trials complete
 40% of trials complete
 50% of trials complete
 60% of trials complete
 70% of trials complete
 80% of trials complete
 90% of trials complete
100% of trials complete


The code below reports the percentage of successes. Typically, the network has shown 70-80% success after repeated trials.

In [24]:
success = len(np.where(accuracy == 1)[0])
print('Using a single hidden layer with %i nodes, a learning rate of %4.2f and %i epochs,' % (num_hidden_nodes,learning_rate, num_epochs))
print('%i out of %i trials (%4.1f%%) achieved perfect classification accuracy for XOR' % (success, num_trials, success/num_trials * 100))

Using a single hidden layer with 2 nodes, a learning rate of 0.10 and 10000 epochs,
72 out of 100 trials (72.0%) achieved perfect classification accuracy for XOR


As on final test, it is worth exploring how a change in the architecture affects the convergence rate for 100% accuracy. The code below repeats the trials, this time with three nodes in the hidden layer, rather than two.

In [25]:
# Run trials
num_hidden_nodes = 3
learning_rate = 0.1
num_epochs = 10000
num_trials = 100
accuracy = run_trials(num_hidden_nodes,learning_rate,num_epochs,num_trials)

 10% of trials complete
 20% of trials complete
 30% of trials complete
 40% of trials complete
 50% of trials complete
 60% of trials complete
 70% of trials complete
 80% of trials complete
 90% of trials complete
100% of trials complete


In [26]:
success = len(np.where(accuracy == 1)[0])
print('Using a single hidden layer with %i nodes, a learning rate of %4.2f and %i epochs,' % (num_hidden_nodes,learning_rate, num_epochs))
print('%i out of %i trials (%4.1f%%) achieved perfect classification accuracy for XOR' % (success, num_trials, success/num_trials * 100))

Using a single hidden layer with 3 nodes, a learning rate of 0.10 and 10000 epochs,
98 out of 100 trials (98.0%) achieved perfect classification accuracy for XOR


By adding another hidden node, we increased the 100% accuracy rate from 72% to 98%. This is a huge improvement, and it was gained by adding a bit more complexity to the model.

## Conceptual Overview

The purpose of this assignment was to complete the creation of our own neural network library. The library is flexible enough to create an arbitrary number of sequential layers with different numbers of nodes and activation functions for each. The resulting network could be used for either binary classification (using a sigmoid activation function for the last layer), or regression (using a linear activation function for the last layer). The model is first trained by passing each row of the training data tensor through the network, producing an output after the final layer. This output is compared to the target value from the corresponding row in the training label tensor, and the weights of the network are updated using the backpropagation algorithm. The backpropagation algorithm updates each weight using the gradient of the error term with respect to the weight, and the magnitude of this update is controlled using the learning rate, which must be positive and has a maximum value of one. This process is repeated for the entire training data tensor, and for the number of epochs requested.

Overall, the results of this assignment were very encouraging. Given the skeleton of the library, it took only a few hours to build up a functional program for creating our own neural networks. As with the Keras example from last week, the network does not achieve 100% accuracy on the XOR data every time it is trained. I thought it would be worth exploring the convergence rate, and ran the series of trials described above. The fact that the convergence rate for 100% accuracy is typically 70-80% is encouraging, but a higher rate (ideally over 90%) would be ideal. After conducting a bit of research, it seems the best way to increase the rate is to either add nodes to the single hidden layer, or add another layer all together. This seems to have the intended effect of pushing the convergence rate to over 90% in a lot of cases. I tried adding a third hidden node, and the convergence rate jumped to 98%, which is quite remarkable. I imagine these types of exercises will be common place once we start building more complex networks, and there isn't a preset formula for how the network should be constructed. Nevertheless, the fact that we can create a network capable of separating nonlinear classification data with just a few dozen lines of code is very impressive.